In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from PIL import Image
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import random
import datetime
import os

In [ ]:
# print(os.listdir("../input/imet-2019-fgvc6/train"))

In [ ]:
train = pd.read_csv("../input/imet-2019-fgvc6/train.csv")
lable = pd.read_csv("../input/imet-2019-fgvc6/labels.csv")

In [ ]:
lable_length = len(lable)
train_length = len(train)
train_length
lable_length

In [ ]:
np.array(train)

In [ ]:
def creatData(train,lable_length):
    train = np.array(train)
    train_data = []
    for t in range(train_length):
        v = np.zeros(lable_length)
        #print(train[t,1])
        for s in train[t,1].split(" "):
            #print(s)
            v[int(s)] = 1
        train_data.append([train[t,0],v])
    return np.array(train_data)

In [ ]:
train_lib = creatData(train,lable_length)
print(train_lib)

In [ ]:
train_transformer = transforms.Compose([
  transforms.Resize((128,128)),              # resize the image to 
  transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
  transforms.ToTensor()])             # transform it into a PyTorch Tensor

In [ ]:
class trainDataset(Dataset):
    def __init__(self, train_lib, transform=None):
        self.filenames = train_lib[:,0]
        self.labels = train_lib[:,1]
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open("../input/imet-2019-fgvc6/train/"+format(self.filenames[idx])+'.png')  # PIL image
        image = self.transform(image)
        return image, self.labels[idx]


In [ ]:
train_dataloader = DataLoader(trainDataset(train_lib, train_transformer), 
                              batch_size=32, shuffle=True)

In [ ]:
(np.array(train_dataloader.dataset[4][0]).shape)

In [ ]:
class InceptionA(nn.Module):
    def __init__(self,in_channels):
        super(InceptionA, self).__init__()
        self.conv1 = nn.Sequential(       # input  (3,_,_)
            nn.AvgPool2d(3,1,1),          # output  (3,_,_)
            nn.Conv2d(in_channels,16,1),  # output  (16,_,_)
        )
        self.conv2 = nn.Sequential(       # input  (3,_,_)
            nn.MaxPool2d(3,1,1),          # output  (3,_,_)
            nn.Conv2d(in_channels,16,1),  # output  (16,_,_)
        )
        
        self.conv3 = nn.Sequential(       # input  (3,_,_)
            nn.Conv2d(in_channels,16,1),  # output  (16,_,_)
            nn.Conv2d(16,16,5,1,2),       # output  (16,_,_)
        )
        
        self.conv4 = nn.Sequential(       # input  (3,_,_)
            nn.Conv2d(in_channels,8,1),   # output  (8,_,_)
            #nn.Conv2d(16,24,3,1,1),      # output  (24,_,_)
            nn.Conv2d(8,16,3,1,1),        # output  (16,_,_)
        )
        
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x4 = self.conv4(x)
        
        outputs = [x2,x3,x4,x1]
        #outputs = [x3,x4,x1]#72
        
        return torch.cat(outputs,1)       # output  (16*4, 32, 32)<<
                                          # output  (88, 32, 32)<<
####################################################################
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(       # input  (3, 32, 32)
            nn.Conv2d(3,20,5,1,2),        # output  (20, 32, 32)
            InceptionA(in_channels=20),   # output  (64, 32, 32)
            nn.MaxPool2d(4),              # output  (64, 16, 16)
        )
        self.conv2 = nn.Sequential(       # input  (64, 16, 16)
            nn.Conv2d(64,40,5,1,2),       # output  (40, 16, 16)
            InceptionA(in_channels=40),   # output  (64, 16, 16)
            nn.MaxPool2d(2),              # output  (64, 8, 8)
        )
        
        self.out1 = nn.Linear(64*16*16, 64*16*8) #fully connected layer
        self.out2 = nn.Linear(64*16*8, 64*16*4)
        self.out3 = nn.Linear(64*16*4, 1103)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #x = x.view(-1, 32 * 8 * 8)
        x = x.view(x.size(0), -1)
        x = self.out1(x)
        x = self.out2(x)
        x = self.out3(x)
        #output = nn.functional.log_softmax(x, dim=1)
        return x    # return x for visualization

In [ ]:
cnn = CNN()
cnn.cuda()
print(cnn)

In [ ]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()

In [ ]:
def train(epoch):
    for step, (x, y) in enumerate(train_dataloader):
        data = Variable(x).cuda()   # batch x
        target = Variable(y).cuda()   # batch y
        #print(data.type())
        #print(target.type())
        output = cnn(data)               # cnn output
        #loss = nn.functional.nll_loss(output, target)
        loss = loss_func(output, target.float())   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, 
           step * len(data), len(train_dataloader.dataset),100. * step / len(train_dataloader), loss.data.item()))
    print("Finish")

In [ ]:
for epoch in range(2):
    train(epoch)

In [ ]:
torch.save(cnn, 'net.pkl')